# 

- **Questions**: [Here](../data/homework_1/HW.pdf)
- **Answer Set** : NO. 01
- **Full Name** : Fatemeh Karimi Barikarasfi
- **Student Code** : 610301060

The goal is Sentiment Analysis using NGram Language Model, the dataset is [IMDB](https://drive.google.com/drive/folders1F88DpdjwHRAfFko0Z4J_SnRas2PYRU4j) which contains people ideas and indicate the aspect of the idea which means that is the idea positive or negative.

## Importing Needed Libraries

In [12]:
import nltk
import pickle
import numpy

## importing data

Load train, validation, and test data:

In [13]:
with open('x_train.pickle', 'rb') as f:
    x_train = pickle.load(f)

with open('x_test.pickle', 'rb') as f:
    x_test = pickle.load(f)

with open('x_val.pickle', 'rb') as f:
    x_val = pickle.load(f)


y_train = numpy.loadtxt('y_train.txt', dtype='int32')
y_val = numpy.loadtxt('y_val.txt', dtype='int32')

## preprocessing 

Do removing unwanted digits, unwanted punchuations, lowering all characters, word tokenization, removing stopwords, and stemming as preprocessing:

In [18]:
unwanted_digit=['0','1','2','3','4','5','6','7','8','9']
unwanted_punc=['<br />', '...',',','"','=','@','&','%',',',':','\\','$','^','<','>','{','}',';','\n','\t','(',')','[',']','/','*','+','#','-','_','|']
# removing unwanted digits
for digit in unwanted_digit:
    x_train = [sent.replace(digit, "") for sent in x_train]
    x_test = [sent.replace(digit, "") for sent in x_test]
    x_val = [sent.replace(digit, "") for sent in x_val]
# removing unwanted punchuations
for punc in unwanted_punc:
    x_train = [sent.replace(punc, "") for sent in x_train]
    x_test = [sent.replace(punc, "") for sent in x_test]
    x_val = [sent.replace(punc, "") for sent in x_val]
# lowering all characters
x_train = [sent.lower() for sent in x_train]
x_test = [sent.lower() for sent in x_test]
x_val = [sent.lower() for sent in x_val]
# word tokenization
x_train = [nltk.word_tokenize(sent) for sent in x_train]
x_test = [nltk.word_tokenize(sent) for sent in x_test]
x_val = [nltk.word_tokenize(sent) for sent in x_val]
# removing stopwords
from nltk.corpus import stopwords
stop_words = stopwords.words("english")
for i in range(len(x_train)):
    x_train[i] = [w for w in x_train[i] if w not in stop_words]
for i in range(len(x_test)):
    x_test[i] = [w for w in x_test[i] if w not in stop_words]
for i in range(len(x_val)):
    x_val[i] = [w for w in x_val[i] if w not in stop_words]
# stemming
from nltk.stem import PorterStemmer
stemmer=PorterStemmer()
for i in range(len(x_train)):
    x_train[i] = [stemmer.stem(w) for w in x_train[i]]
for i in range(len(x_test)):
    x_test[i] = [stemmer.stem(w) for w in x_test[i]]
for i in range(len(x_val)):
    x_val[i] = [stemmer.stem(w) for w in x_val[i]]


## Preparing data for trainig and testing models

In [25]:
from nltk.lm.preprocessing import flatten

x_train_positive = nltk.sent_tokenize(" ".join(flatten([x_train[i] for i in range(0, len(x_train)) if y_train[i] == 1])))
x_train_negative = nltk.sent_tokenize(" ".join(flatten([x_train[i] for i in range(0, len(x_train)) if y_train[i] == 0])))

x_val_sents = [" ".join(item) for item in x_val]
x_val_positive_sents = [x_val_sents[i] for i in range(len(x_val_sents)) if y_val[i] == 1]
x_val_negative_sents = [x_val_sents[i] for i in range(len(x_val_sents)) if y_val[i] == 0]

x_test_sents = [" ".join(item) for item in x_test]

## Models

Create NGrams Models with N = 1, 2, 3, 4 

<div style="direction:rtl"> در این مرحله مدل های گفته شده در صورت سوال ، پیاده سازی شده است . برای فرآیند smoothing از تکنیک laplace استفاده شده است . </div>

In [30]:
from nltk.lm import Laplace
from nltk.lm.preprocessing import padded_everygram_pipeline

## unigram models

In [31]:
uni_model_positive = Laplace(1)
uni_model_positive.fit(*padded_everygram_pipeline(1, x_train_positive))

uni_model_negative = Laplace(1)
uni_model_negative.fit(*padded_everygram_pipeline(1, x_train_negative))

## bigram models

In [33]:
bi_model_positive = Laplace(2)
bi_model_positive.fit(*padded_everygram_pipeline(2, x_train_positive))

bi_model_negative = Laplace(2)
bi_model_negative.fit(*padded_everygram_pipeline(2, x_train_negative))

## trigram model

In [34]:
tri_model_positive = Laplace(3)
tri_model_positive.fit(*padded_everygram_pipeline(3, x_train_positive))

tri_model_negative = Laplace(3)
tri_model_negative.fit(*padded_everygram_pipeline(3, x_train_negative))

## quadgram model

In [35]:
quad_model_positive = Laplace(4)
quad_model_positive.fit(*padded_everygram_pipeline(4, x_train_positive))

quad_model_negative = Laplace(4)
quad_model_negative.fit(*padded_everygram_pipeline(4, x_train_negative))

# 3.

<div style="direction:rtl"> در این قسمت بیشترین ngram تکرار شده در هر مدل نشان داده شده است . </div>

In [36]:
from nltk.util import ngrams
from nltk.probability import FreqDist

In [37]:
for i in range(len(x_train)):
    x_train[i] = [w for w in x_train[i] if w not in ['.', '!', '?']]

In [38]:
x_p = list(flatten([x_train[i] for i in range(len(x_train)) if y_train[i] == 1]))
x_n = list(flatten([x_train[i] for i in range(len(x_train)) if y_train[i] == 0]))

## most frequent n-gram of each models 

### unigram

#### positive feelings

In [39]:
unigram_list_p = list(ngrams(x_p, 1))
freq_dist_uni_p = FreqDist(unigram_list_p)

In [40]:
freq_dist_uni_p

FreqDist({("'s",): 31422, ('film',): 24453, ('movi',): 21663, ('one',): 13605, ("n't",): 13366, ('like',): 10067, ('time',): 7804, ('good',): 7391, ('see',): 7201, ('charact',): 6915, ...})

#### negative feelings

In [41]:
unigram_list_n = list(ngrams(x_n, 1))
freq_dist_uni_n = FreqDist(unigram_list_n)

In [42]:
freq_dist_uni_n

FreqDist({("'s",): 29613, ('movi',): 28004, ('film',): 21520, ("n't",): 19946, ('one',): 13018, ('like',): 12112, ('get',): 7700, ('make',): 7694, ('even',): 7671, ('would',): 7661, ...})

### bigram

#### positive feelings

In [43]:
bigram_list_p = list(ngrams(x_p, 2))
freq_dist_bi_p = FreqDist(bigram_list_p)

In [44]:
freq_dist_bi_p

FreqDist({('ca', "n't"): 1436, ('film', "'s"): 910, ('one', 'best'): 801, ('watch', 'movi'): 662, ("'ve", 'seen'): 648, ('wo', "n't"): 611, ("n't", 'know'): 554, ('movi', "'s"): 523, ('even', 'though'): 513, ('look', 'like'): 491, ...})

#### negative feelings

In [45]:
bigram_list_n = list(ngrams(x_n, 2))
freq_dist_bi_n = FreqDist(bigram_list_n)

In [46]:
freq_dist_bi_n

FreqDist({('ca', "n't"): 2081, ('look', 'like'): 1399, ("n't", 'even'): 1150, ('could', "n't"): 1063, ('watch', 'movi'): 860, ("n't", 'know'): 822, ('wast', 'time'): 813, ('ever', 'seen'): 810, ('film', "'s"): 775, ('special', 'effect'): 721, ...})

### trigram

#### positive feelings

In [47]:
trigram_list_p = list(ngrams(x_p, 3))
freq_dist_tri_p = FreqDist(trigram_list_p)

In [48]:
freq_dist_tri_p

FreqDist({("'ve", 'ever', 'seen'): 154, ('ca', "n't", 'help'): 108, ('new', 'york', 'citi'): 101, ('ca', "n't", 'wait'): 99, ('film', "'ve", 'seen'): 90, ('film', 'ever', 'made'): 84, ('one', 'best', 'movi'): 82, ('world', 'war', 'ii'): 78, ('base', 'true', 'stori'): 69, ('one', 'best', 'film'): 69, ...})

#### negative feelings

In [49]:
trigram_list_n = list(ngrams(x_n, 3))
freq_dist_tri_n = FreqDist(trigram_list_n)

In [50]:
freq_dist_tri_n

FreqDist({("'ve", 'ever', 'seen'): 324, ('worst', 'movi', 'ever'): 259, ('ca', "n't", 'believ'): 193, ("n't", 'wast', 'time'): 189, ('movi', 'ever', 'seen'): 189, ('one', 'worst', 'movi'): 149, ('worst', 'film', 'ever'): 134, ('movi', "'ve", 'ever'): 128, ('ca', "n't", 'even'): 110, ('worst', 'movi', "'ve"): 109, ...})

### quadgram

#### positive feelings

In [51]:
quadgram_list_p = list(ngrams(x_p, 4))
freq_dist_quad_p = FreqDist(quadgram_list_p)

In [52]:
freq_dist_quad_p

FreqDist({('ca', "n't", 'wait', 'see'): 38, ('movi', "'ve", 'ever', 'seen'): 29, ('film', "'ve", 'ever', 'seen'): 25, ("'ve", 'seen', 'long', 'time'): 19, ('one', 'best', 'movi', 'ever'): 18, ('one', 'best', 'movi', "'ve"): 17, ('best', 'movi', 'ever', 'made'): 15, ('ca', "n't", 'help', 'feel'): 15, ('ca', "n't", 'go', 'wrong'): 14, ('best', 'movi', "'ve", 'seen'): 14, ...})

#### negative feelings

In [53]:
quadgram_list_n = list(ngrams(x_n, 4))
freq_dist_quad_n = FreqDist(quadgram_list_n)

In [54]:
freq_dist_quad_n

FreqDist({('worst', 'movi', 'ever', 'seen'): 127, ('movi', "'ve", 'ever', 'seen'): 105, ('one', 'worst', 'movi', 'ever'): 77, ('worst', 'movi', "'ve", 'ever'): 75, ('worst', 'film', 'ever', 'seen'): 61, ('worst', 'movi', 'ever', 'made'): 51, ('film', "'ve", 'ever', 'seen'): 48, ('one', 'worst', 'film', 'ever'): 43, ('worst', 'film', 'ever', 'made'): 39, ('one', 'worst', 'movi', "'ve"): 39, ...})

# 4.

<div style="direction:rtl"> در این قسمت perplexity هر مدل محاسبه شده است . </div>

## perplexity of each models

### unigram

#### positive feelings

In [55]:
uni_model_positive.perplexity(x_val_positive_sents)

86.99999999999875

#### negative feelings

In [56]:
uni_model_negative.perplexity(x_val_negative_sents)

84.99999999999947

### bigram

#### positive feelings

In [57]:
bi_model_positive.perplexity(x_val_positive_sents)

89.0000000000047

#### negative feelings

In [58]:
bi_model_negative.perplexity(x_val_negative_sents)

86.99999999999875

### trigram

#### positive feelings

In [59]:
tri_model_positive.perplexity(x_val_positive_sents)

89.0000000000047

#### negative feelings

In [60]:
tri_model_negative.perplexity(x_val_negative_sents)

86.99999999999875

### quadgram

#### positive feelings

In [61]:
quad_model_positive.perplexity(x_val_positive_sents)

89.0000000000047

#### negative feelings

In [62]:
quad_model_negative.perplexity(x_val_negative_sents)

86.99999999999875

# 5.

<div style="direction:rtl"> در این قسمت مدل های پیاده سازی شده ارزیابی شده است و دقت هرکدام print  شده است . </div>

<div style="direction:rtl"> برای تشخیص کلاس هر نظر از معیار perplexity استفاده شده است زیرا این معیار متناسب با معکوس احتمال جمله می باشد و با توجه به این میتوان گفت هر مدلی که perplexity کمتری برای جمله داشته باشد آن جمله به کلاس آن مدل تعلق دارد . . </div>

## evaluating each models

### unigram

In [63]:
uni_positive_perplexities = [uni_model_positive.perplexity(sent) for sent in x_val_sents]
uni_negative_perplexities = [uni_model_negative.perplexity(sent) for sent in x_val_sents]

In [64]:
uni_labels = [1 if uni_positive_perplexities[i] < uni_negative_perplexities[i] else 0 for i in range(len(y_val))]

In [65]:
succs = 0
for i in range(len(uni_labels)):
    if uni_labels[i] == y_val[i]:
        succs = succs + 1

print(succs * 100 / len(uni_labels))

62.384


### bigram

In [66]:
bi_positive_perplexities = [bi_model_positive.perplexity(sent) for sent in x_val_sents]
bi_negative_perplexities = [bi_model_negative.perplexity(sent) for sent in x_val_sents]

In [67]:
bi_labels = [1 if bi_positive_perplexities[i] < bi_negative_perplexities[i] else 0 for i in range(len(y_val))]

In [68]:
succs = 0
for i in range(len(bi_labels)):
    if bi_labels[i] == y_val[i]:
        succs = succs + 1

print(succs * 100 / len(bi_labels))

57.352


### trigram

In [69]:
tri_positive_perplexities = [tri_model_positive.perplexity(sent) for sent in x_val_sents]
tri_negative_perplexities = [tri_model_negative.perplexity(sent) for sent in x_val_sents]

In [70]:
tri_labels = [1 if tri_positive_perplexities[i] < tri_negative_perplexities[i] else 0 for i in range(len(y_val))]

In [71]:
succs = 0
for i in range(len(tri_labels)):
    if tri_labels[i] == y_val[i]:
        succs = succs + 1

print(succs * 100 / len(tri_labels))

52.792


### quadgram

In [72]:
quad_positive_perplexities = [quad_model_positive.perplexity(sent) for sent in x_val_sents]
quad_negative_perplexities = [quad_model_negative.perplexity(sent) for sent in x_val_sents]

In [73]:
quad_labels = [1 if quad_positive_perplexities[i] < quad_negative_perplexities[i] else 0 for i in range(len(y_val))]

In [74]:
succs = 0
for i in range(len(quad_labels)):
    if quad_labels[i] == y_val[i]:
        succs = succs + 1

print(succs * 100 / len(quad_labels))

51.112


# 6.

<div style="direction:rtl"> با توجه به ارزیابی قسمت قبل بهترین مدل ، مدل unigram می باشد . </div>

In [75]:
uni_positive_perplexities_test = [uni_model_positive.perplexity(sent) for sent in x_test_sents]
uni_negative_perplexities_test = [uni_model_negative.perplexity(sent) for sent in x_test_sents]

In [76]:
test_labels = [1 if uni_positive_perplexities_test[i] < uni_negative_perplexities_test[i] else 0 for i in range(len(x_test_sents))]

In [79]:
numpy.savetxt("y_test.csv", test_labels)